<a href="https://colab.research.google.com/github/mounsifelatouch/cdd/blob/master/notebooks/3_cdd_ml_part_3_descriptor_dataset_preparation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Bioinformatics Project - Computational Drug Discovery [Part 3] Descriptor Calculation and Dataset Preparation**

**MOUNSIF EL ATOUCH**

In this Jupyter notebook, we will be building a machine learning model using the ChEMBL bioactivity data.

In **Part 3**, we will be calculating molecular descriptors that are essentially quantitative description of the compounds in the dataset. Finally, we will be preparing this into a dataset for subsequent model building in Part 4.

---

## **Download PaDEL-Descriptor**

In [1]:
! wget https://raw.githubusercontent.com/mounsifelatouch/cdd/master/padel/padel.zip
! wget https://raw.githubusercontent.com/mounsifelatouch/cdd/master/padel/padel.sh

--2023-06-01 03:59:32--  https://raw.githubusercontent.com/mounsifelatouch/cdd/master/padel/padel.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.111.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 25768637 (25M) [application/zip]
Saving to: ‘padel.zip’

padel.zip           100%[===================>]  24.57M   142MB/s    in 0.2s    

2023-06-01 03:59:35 (142 MB/s) - ‘padel.zip’ saved [25768637/25768637]

--2023-06-01 03:59:35--  https://raw.githubusercontent.com/mounsifelatouch/cdd/master/padel/padel.sh
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 233 [text/plain]
Saving to:

In [2]:
! unzip padel.zip

Archive:  padel.zip
   creating: PaDEL-Descriptor/
  inflating: __MACOSX/._PaDEL-Descriptor  
  inflating: PaDEL-Descriptor/MACCSFingerprinter.xml  
  inflating: __MACOSX/PaDEL-Descriptor/._MACCSFingerprinter.xml  
  inflating: PaDEL-Descriptor/AtomPairs2DFingerprinter.xml  
  inflating: __MACOSX/PaDEL-Descriptor/._AtomPairs2DFingerprinter.xml  
  inflating: PaDEL-Descriptor/EStateFingerprinter.xml  
  inflating: __MACOSX/PaDEL-Descriptor/._EStateFingerprinter.xml  
  inflating: PaDEL-Descriptor/Fingerprinter.xml  
  inflating: __MACOSX/PaDEL-Descriptor/._Fingerprinter.xml  
  inflating: PaDEL-Descriptor/.DS_Store  
  inflating: __MACOSX/PaDEL-Descriptor/._.DS_Store  
   creating: PaDEL-Descriptor/license/
  inflating: __MACOSX/PaDEL-Descriptor/._license  
  inflating: PaDEL-Descriptor/KlekotaRothFingerprintCount.xml  
  inflating: __MACOSX/PaDEL-Descriptor/._KlekotaRothFingerprintCount.xml  
  inflating: PaDEL-Descriptor/config  
  inflating: __MACOSX/PaDEL-Descriptor/._config  
  inf

## **Installing librairies**

In [3]:
! pip install rdkit

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 29.7/29.7 MB 24.8 MB/s eta 0:00:00


## **Importing libraires**

In [4]:
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

## **Load bioactivity data**

Download the curated ChEMBL bioactivity data that has been pre-processed from Parts 1 and 2 of this Bioinformatics Project series. Here we will be using the **bioactivity_data_curated.csv** file that essentially contain the pIC50 values that we will be using for building a classification model.

In [5]:
! wget https://raw.githubusercontent.com/mounsifelatouch/cdd/master/data/bioactivity_data_curated.csv

--2023-06-01 03:59:47--  https://raw.githubusercontent.com/mounsifelatouch/cdd/master/data/bioactivity_data_curated.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.111.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 54330 (53K) [text/plain]
Saving to: ‘bioactivity_data_curated.csv’

bioactivity_data_cu 100%[===================>]  53.06K  --.-KB/s    in 0.01s   

2023-06-01 03:59:48 (4.09 MB/s) - ‘bioactivity_data_curated.csv’ saved [54330/54330]



In [6]:
df = pd.read_csv('bioactivity_data_curated.csv')

In [7]:
selection = ['chembl_id', 'canonical_smiles']
df_selection = df[selection]
df_selection.to_csv('molecule.smi', sep='\t', index=False, header=False)

In [8]:
! cat molecule.smi | head -5

CHEMBL1466	O=c1oc2ccccc2c(O)c1Cc1c(O)c2ccccc2oc1=O
CHEMBL260998	O=c1oc2ccccc2c(O)c1C(c1ccc[nH]1)c1c(O)c2ccccc2oc1=O
CHEMBL260997	CCOc1cc(C(c2c(O)c3ccccc3oc2=O)c2c(O)c3ccccc3oc2=O)ccc1O
CHEMBL258733	O=c1oc2ccccc2c(O)c1C(c1c(O)c2ccccc2oc1=O)c1c[nH]c2ccccc12
CHEMBL81935	O=c1c(C(c2ccc([N+](=O)[O-])cc2)c2c(O)oc3ccccc3c2=O)c(O)oc2ccccc12


In [9]:
! cat molecule.smi | wc -l

648


## **Calculate fingerprint descriptors**


### **Calculate PaDEL descriptors**

In [10]:
! cat padel.sh

java -Xms1G -Xmx1G -Djava.awt.headless=true -jar ./PaDEL-Descriptor/PaDEL-Descriptor.jar -removesalt -standardizenitro -fingerprints -descriptortypes ./PaDEL-Descriptor/GraphOnlyFingerprinter.xml -dir ./ -file descriptors_output.csv


In [11]:
%%time
! bash padel.sh

Descriptor calculation completed in 0.392 secs . Average speed: ∞ s/mol.
CPU times: user 27.4 ms, sys: 6.93 ms, total: 34.3 ms
Wall time: 3.02 s


## **Preparing the X and Y Data Matrices**

### **X data matrix**

In [12]:
df_X = pd.read_csv('descriptors_output.csv')

In [13]:
df_X = df_X.drop(columns=['Name'])

In [14]:
df_X

,GraphFP1,GraphFP2,GraphFP3,GraphFP4,GraphFP5,GraphFP6,GraphFP7,GraphFP8,GraphFP9,GraphFP10,...,GraphFP1015,GraphFP1016,GraphFP1017,GraphFP1018,GraphFP1019,GraphFP1020,GraphFP1021,GraphFP1022,GraphFP1023,GraphFP1024


## **Y variable**

In [15]:
df_Y = df['target']

In [16]:
df_Y

0      inactive
1      inactive
2      inactive
3      inactive
4      inactive
         ...   
643    inactive
644    inactive
645    inactive
646    inactive
647    inactive
Name: target, Length: 648, dtype: object

## **Combining X and Y variable**

In [17]:
data = pd.concat([df_X, df_Y], axis=1)

In [18]:
data

,GraphFP1,GraphFP2,GraphFP3,GraphFP4,GraphFP5,GraphFP6,GraphFP7,GraphFP8,GraphFP9,GraphFP10,...,GraphFP1016,GraphFP1017,GraphFP1018,GraphFP1019,GraphFP1020,GraphFP1021,GraphFP1022,GraphFP1023,GraphFP1024,target
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,inactive
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,inactive
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,inactive
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,inactive
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,inactive
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
643,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,inactive
644,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,inactive
645,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,inactive
646,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,inactive


In [19]:
data.to_csv('bioactivity_data_pubchem_fp.csv', index=False)

In [20]:
! zip lab3.zip *

  adding: bioactivity_data_curated.csv (deflated 77%)
  adding: bioactivity_data_pubchem_fp.csv (deflated 99%)
  adding: descriptors_output.csv (deflated 81%)
  adding: __MACOSX/ (stored 0%)
  adding: molecule.smi (deflated 82%)
  adding: PaDEL-Descriptor/ (stored 0%)
  adding: padel.sh (deflated 36%)
  adding: padel.zip (stored 0%)
  adding: sample_data/ (stored 0%)
